# change the data information

In [2]:
# go to file : lib/dataset/datasets/coco_custom.py
# change following two information
# it is usually not be changed
# but the small data contain only the following four classes (from notebook 1_convert_the_data)

class_name = ['car', 'truck', 'van', 'bus']

_valid_ids = [1,2,3,4]

# Make the search space File network_config.yaml

In [6]:
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import random
import copy
import math
from lib.model.networks.OneForAll_NAS import OneForAll_SuperNetNetwork,rep_model_convert
import yaml
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:

# The size of the search space can be determined by checking at the size of the largest network / inference time

network_config = {
    "backbone" : [
        # block NAS_REP_ConvBNBlock  [ filternumer, stride, run_time_depth_index]
        ['NAS_REP_ConvBNBlock', [96, 1, 0]],
        ['NAS_REP_ConvBNBlock', [128, 2, 0]],
        # block branch_block  [ filternumer, number of repetition blocks, run_time_depth_index]
        ['branch_block',         [128, 2, 1]],
        ['NAS_REP_ConvBNBlock', [256, 2, 0]],
        ['branch_block', [256, 4, 1]],
        ['NAS_REP_ConvBNBlock', [512, 2, 0]],
        ['branch_block', [512, 4, 1]],
        ['NAS_REP_ConvBNBlock', [1024, 2, 0]],
        ['branch_block', [1024, 2, 1]],
        ['NAS_REP_ConvBNBlock', [1024, 1, 0]]
    ],
    "neck" : [
        [[1,2,1], [1,2,1], [1,2,1]],
        [[1,2,1], [1,2,1]],
        [[1,2,1]],
        [1,2,1],
        [1,4,1]
    ],
    "head" : {
        'hm':  3,
        'reg': 3,
        'wh':  3,
        'tracking': 3
    }

}


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


opt = dotdict()

opt.max_kernel_size     = 5
opt.width_mult          = 0.2
opt.pre_img             = True
opt.combine_style       = "sub_cat"
opt.verbose             = False
opt.scaling_factor      = 0.5
opt.skip_connection     = True
num_cls                 = len(class_name)
opt.heads               = {'hm': num_cls, 'reg': 2, 'wh': 2, 'tracking':2}


op = OneForAll_SuperNetNetwork(network_config = network_config, opt=opt).to(device)  


img_size        = 640

x2 = torch.randn([1, 3, img_size, img_size]).to(device)  
x1 = torch.randn([1, 3, img_size, img_size]).to(device)  


op.eval()


op_rep= rep_model_convert(op, do_copy=False)


torch.onnx.export(op_rep,  # model being run
                  (x1,x2) ,
                  "largest_to_delete.onnx",
                  export_params=True,  # store the trained parameter weights inside the model file
                  # keep_initializers_as_inputs=True,
                  # the ONNX version to export the model to
                  opset_version=10,
                  verbose=False,
                  input_names=['input1','input2'],  # the model's input names
                  output_names=['output'],  # the model's output names
                  #dynamic_axes={"input": {0: "batch", 1:"channel",2: "width",3:"height"}}
                 )   



In [8]:
with open('network_config.yaml', 'w') as file:
    yaml.dump(network_config, file, default_flow_style=False)

# Train the SuperNets

In [ ]:
python main.py tracking \
    --exp_id oneforalltest1 \
    --tracking \
    --dataset syntra \
    --flip 0.0 \
    --gpus 0 \
    --batch_size 6 \
    --lr 4e-4 \
    --same_aug \
    --num_workers 16 \
    --shift 0.05 \
    --scale 0.05 \
    --hm_disturb 0.05 \
    --lost_disturb 0.4 \
    --fp_disturb 0.1 \
    --arch oneforall \
    --num_epochs 100 \
    --data_fold_name notebook_data \
    --input_h  800 \
    --input_w  800 \
    --down_ratio 4 \
    --width_mult 0.2

# exp_id you can give anyname 
# --tracking  the task name , if you want to do detection only , --cdet
# batch_size
# arch --> oneforall
# num_epochs  training epoches
# data_fold_name notebook_data
# input_h 



## retrain the model from last break point

In [ ]:
python main.py tracking \
    --exp_id oneforalltest \
    --tracking \
    --dataset syntra \
    --flip 0.0 \
    --gpus 0 \
    --batch_size 16 \
    --lr 4e-4 \
    --same_aug \
    --num_workers 16 \
    --shift 0.05 \
    --scale 0.05 \
    --hm_disturb 0.05 \
    --lost_disturb 0.4 \
    --fp_disturb 0.1 \
    --arch oneforall \
    --num_epochs 100 \
    --data_fold_name notebook_data \
    --input_h  800 \
    --input_w  800 \
    --down_ratio 4 \
    --load_model /pfs/data5/home/kit/tm/px6680/Conference/Syntra/exp/tracking/oneforalltest/model_last.pth \

Inference

In [ ]:
python demo.py tracking \
    --load_model /pfs/data5/home/kit/tm/px6680/Conference/Syntra/exp/tracking/oneforalltest/model_last.pth \
    --arch oneforall \
    --dataset syntra \
    --data_fold_name notebook_data \
    --demo /pfs/data5/home/kit/tm/px6680/Conference/WACV/CenterTrack/imgs7 \
    --debug 4 \
    --track_thresh 0.2 \